# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 28 2022 11:33AM,242980,10,MTR-22-0240,"Senseonics, Incorporated",Released
1,Jul 28 2022 11:19AM,242979,10,MSP212627,"Methapharm, Inc.",Released
2,Jul 28 2022 11:19AM,242979,10,MSP212630,"Methapharm, Inc.",Released
3,Jul 28 2022 11:19AM,242979,10,MSP212631,"Methapharm, Inc.",Released
4,Jul 28 2022 11:19AM,242979,10,MSP212633,"Methapharm, Inc.",Released
5,Jul 28 2022 11:19AM,242979,10,MSP212634,"Methapharm, Inc.",Released
6,Jul 28 2022 11:19AM,242979,10,MSP212635,"Methapharm, Inc.",Released
7,Jul 28 2022 11:19AM,242979,10,MSP212636,"Methapharm, Inc.",Released
8,Jul 28 2022 11:19AM,242979,10,MSP212637,"Methapharm, Inc.",Released
9,Jul 28 2022 11:19AM,242979,10,MSP212638,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
51,242974,Released,2
52,242975,Released,8
53,242976,Released,1
54,242979,Released,12
55,242980,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242974,NaN,NaN,2.0
242975,NaN,NaN,8.0
242976,NaN,NaN,1.0
242979,NaN,NaN,12.0
242980,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242855,0.0,1.0,0.0
242856,0.0,0.0,1.0
242857,0.0,0.0,1.0
242858,0.0,0.0,1.0
242860,1.0,2.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242855,0,1,0
242856,0,0,1
242857,0,0,1
242858,0,0,1
242860,1,2,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242855,0,1,0
1,242856,0,0,1
2,242857,0,0,1
3,242858,0,0,1
4,242860,1,2,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242855,,1,
1,242856,,,1
2,242857,,,1
3,242858,,,1
4,242860,1,2,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 28 2022 11:33AM,242980,10,"Senseonics, Incorporated"
1,Jul 28 2022 11:19AM,242979,10,"Methapharm, Inc."
13,Jul 28 2022 11:08AM,242972,15,"Person & Covey, Inc."
21,Jul 28 2022 11:08AM,242975,15,"Virtus Pharmaceuticals, LLC"
29,Jul 28 2022 11:07AM,242974,15,"Mizner Bioscience, LLC"
31,Jul 28 2022 11:07AM,242973,15,"Carwin Pharmaceutical Associates, LLC"
32,Jul 28 2022 11:00AM,242976,22,"NBTY Global, Inc."
33,Jul 28 2022 10:58AM,242970,15,"Alliance Pharma, Inc."
50,Jul 28 2022 10:54AM,242971,19,"GUSA Granules USA, Inc."
51,Jul 28 2022 10:51AM,242969,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 28 2022 11:33AM,242980,10,"Senseonics, Incorporated",,,1
1,Jul 28 2022 11:19AM,242979,10,"Methapharm, Inc.",,,12
2,Jul 28 2022 11:08AM,242972,15,"Person & Covey, Inc.",,,8
3,Jul 28 2022 11:08AM,242975,15,"Virtus Pharmaceuticals, LLC",,,8
4,Jul 28 2022 11:07AM,242974,15,"Mizner Bioscience, LLC",,,2
5,Jul 28 2022 11:07AM,242973,15,"Carwin Pharmaceutical Associates, LLC",,,1
6,Jul 28 2022 11:00AM,242976,22,"NBTY Global, Inc.",,,1
7,Jul 28 2022 10:58AM,242970,15,"Alliance Pharma, Inc.",,,17
8,Jul 28 2022 10:54AM,242971,19,"GUSA Granules USA, Inc.",,,1
9,Jul 28 2022 10:51AM,242969,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 11:33AM,242980,10,"Senseonics, Incorporated",1,,
1,Jul 28 2022 11:19AM,242979,10,"Methapharm, Inc.",12,,
2,Jul 28 2022 11:08AM,242972,15,"Person & Covey, Inc.",8,,
3,Jul 28 2022 11:08AM,242975,15,"Virtus Pharmaceuticals, LLC",8,,
4,Jul 28 2022 11:07AM,242974,15,"Mizner Bioscience, LLC",2,,
5,Jul 28 2022 11:07AM,242973,15,"Carwin Pharmaceutical Associates, LLC",1,,
6,Jul 28 2022 11:00AM,242976,22,"NBTY Global, Inc.",1,,
7,Jul 28 2022 10:58AM,242970,15,"Alliance Pharma, Inc.",17,,
8,Jul 28 2022 10:54AM,242971,19,"GUSA Granules USA, Inc.",1,,
9,Jul 28 2022 10:51AM,242969,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 11:33AM,242980,10,"Senseonics, Incorporated",1,,
1,Jul 28 2022 11:19AM,242979,10,"Methapharm, Inc.",12,,
2,Jul 28 2022 11:08AM,242972,15,"Person & Covey, Inc.",8,,
3,Jul 28 2022 11:08AM,242975,15,"Virtus Pharmaceuticals, LLC",8,,
4,Jul 28 2022 11:07AM,242974,15,"Mizner Bioscience, LLC",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 11:33AM,242980,10,"Senseonics, Incorporated",1.0,NaN,NaN
1,Jul 28 2022 11:19AM,242979,10,"Methapharm, Inc.",12.0,NaN,NaN
2,Jul 28 2022 11:08AM,242972,15,"Person & Covey, Inc.",8.0,NaN,NaN
3,Jul 28 2022 11:08AM,242975,15,"Virtus Pharmaceuticals, LLC",8.0,NaN,NaN
4,Jul 28 2022 11:07AM,242974,15,"Mizner Bioscience, LLC",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 11:33AM,242980,10,"Senseonics, Incorporated",1.0,0.0,0.0
1,Jul 28 2022 11:19AM,242979,10,"Methapharm, Inc.",12.0,0.0,0.0
2,Jul 28 2022 11:08AM,242972,15,"Person & Covey, Inc.",8.0,0.0,0.0
3,Jul 28 2022 11:08AM,242975,15,"Virtus Pharmaceuticals, LLC",8.0,0.0,0.0
4,Jul 28 2022 11:07AM,242974,15,"Mizner Bioscience, LLC",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3401194,64.0,22.0,3.0
15,1214864,36.0,0.0,0.0
16,971784,4.0,0.0,0.0
19,1457498,30.0,20.0,14.0
20,1457454,57.0,6.0,9.0
21,971426,3.0,1.0,0.0
22,728913,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3401194,64.0,22.0,3.0
1,15,1214864,36.0,0.0,0.0
2,16,971784,4.0,0.0,0.0
3,19,1457498,30.0,20.0,14.0
4,20,1457454,57.0,6.0,9.0
5,21,971426,3.0,1.0,0.0
6,22,728913,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,64.0,22.0,3.0
1,15,36.0,0.0,0.0
2,16,4.0,0.0,0.0
3,19,30.0,20.0,14.0
4,20,57.0,6.0,9.0
5,21,3.0,1.0,0.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,64.0
1,15,Released,36.0
2,16,Released,4.0
3,19,Released,30.0
4,20,Released,57.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,3.0,0.0,0.0,14.0,9.0,0.0,0.0
Executing,22.0,0.0,0.0,20.0,6.0,1.0,0.0
Released,64.0,36.0,4.0,30.0,57.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,3.0,0.0,0.0,14.0,9.0,0.0,0.0
1,Executing,22.0,0.0,0.0,20.0,6.0,1.0,0.0
2,Released,64.0,36.0,4.0,30.0,57.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,3.0,0.0,0.0,14.0,9.0,0.0,0.0
1,Executing,22.0,0.0,0.0,20.0,6.0,1.0,0.0
2,Released,64.0,36.0,4.0,30.0,57.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()